In [5]:
import sys

import pygame as pg
from pygame.locals import *

from car import Car
from track import Track

import neat
import math
import time

In [6]:
class NeatCar(Car):
    MAX_LENGTH = 400
    
    def __init__(self, car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height):
        super().__init__(car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height)
        self.radars = []
    
    def get_reward(self):
        return self.distance / (Car.CAR_SIZE_X / 2)
        return self.distance / (Car.CAR_SIZE_X / 2) + 1000 / self.time
    
    def check_radar(self, degree):
        length = 0
        x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
        y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
        
        while not self.is_out_of_bounds(x, y) and not self.is_collision((x, y)) and length < NeatCar.MAX_LENGTH:
            length += 1
            x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
            y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
            
        dist = int(pg.math.Vector2(x, y).distance_to(self.center))
        self.radars.append([(x, y), dist])
        
    def draw(self, screen):
        super().draw(screen)
        for r in self.radars:
            pg.draw.line(screen, (0, 255, 0), self.center, r[0], 1)
            pg.draw.circle(screen, (0, 255, 0), r[0], 5)
            
    def update(self):
        super().update()
        self.radars.clear()
        for d in range(-90, 120, 45):
            self.check_radar(d)
            
    def get_neat_data(self):
        return [r[1] // 30 for r in self.radars] if len(self.radars) > 0 else [0, 0, 0, 0, 0]

In [7]:
config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')

population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))

stats = neat.StatisticsReporter()
population.add_reporter(stats)

In [8]:
def run_simulation(genomes, config, map_path='tracks/track01.png', speed = 20, full_screen = True):    
    networks = []
    cars = []
    
    pg.init()
    
    my_track = Track(map_path)
    
    screen = pg.display.set_mode((my_track.map_width, my_track.map_height), (SCALED | RESIZABLE) if not full_screen else FULLSCREEN)
    my_track.load_game_map()
    
    clock = pg.time.Clock()
    
    start_pos_x, start_pos_y, angle = my_track.start_pos
    
    start_pos_y -= Car.CAR_SIZE_Y // 2
    
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        networks.append(net)
        g.fitness = 0
        cars.append(NeatCar('cars/car2d.png', start_pos_x, start_pos_y, angle, speed, my_track.game_map, my_track.border_color, my_track.width, my_track.height))
        
    font_generation = pg.font.SysFont("Arial", 30)
    font_alive = pg.font.SysFont("Arial", 20)
    
    global current_generation
    current_generation += 1
    
    timer = time.time()
    MAX_TIME = 10
    
    while True:
        for event in pg.event.get():
            if event.type == QUIT:
                pg.quit()
                sys.exit()
                
        no_still_alive = 0
        
        for i, car in enumerate(cars):
            output = networks[i].activate(car.get_neat_data())
            choice = output.index(max(output))
            
            if choice == 0:
                car.angle += 10 # Turn left
            elif choice == 1:
                car.angle -= 10 # Turn right
            elif choice == 2:
                if car.speed >= 10:
                    car.speed -= 2 # Slow down
            else:
                car.speed += 2 # Speed up
                
            if car.is_alive():
                no_still_alive += 1
                car.update()
                genomes[i][1].fitness = car.get_reward()
                
        if no_still_alive == 0 or time.time() - timer > MAX_TIME:
            break
        
        screen.blit(my_track.game_map, (0, 0))
        for car in cars:
            if car.is_alive():
                car.draw(screen)
                
        text_gen = font_generation.render("Generation: " + str(current_generation), True, (0, 0, 0))
        text_gen_rect = text_gen.get_rect()
        text_gen_rect.center = (my_track.width // 2, my_track.height // 2)
        screen.blit(text_gen, text_gen_rect)
        
        text_alive = font_alive.render("Alive: " + str(no_still_alive), True, (0, 0, 0))
        text_alive_rect = text_alive.get_rect()
        text_alive_rect.center = (my_track.width // 2, my_track.height // 2 + 50)
        screen.blit(text_alive, text_alive_rect)
        
        pg.display.flip()
        clock.tick(60)
        
    # pg.quit()

current_generation = 0
population.run(run_simulation, 10)
pg.quit()


 ****** Running generation 0 ****** 

Population's average fitness: 24.10222 stdev: 71.08153
Best fitness: 399.66667 - size: (4, 20) - species 1 - id 3
Average adjusted fitness: 0.052
Mean genetic distance 1.106, standard deviation 0.233
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30    399.7    0.052     0
Total extinctions: 0
Generation time: 12.167 sec

 ****** Running generation 1 ****** 

Population's average fitness: 45.45556 stdev: 80.28758
Best fitness: 343.20000 - size: (4, 20) - species 1 - id 3
Average adjusted fitness: 0.124
Mean genetic distance 1.212, standard deviation 0.299
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    30    343.2    0.124     1
Total extinctions: 0
Generation time: 12.002 sec (12.084 average)

 ****** Running generation 2 ****** 

Population's average fitness: 47.93778

SystemExit: 